In [1]:
from collections import namedtuple

from batcher import Batcher
from data import Vocab

from absl import flags
from extractor import run_selector
from extractor.model import ExtractorModel

flags.FLAGS(["extractor"])
from utils import settings

FLAGS = flags.FLAGS

In [2]:
VOCAB_PATH= '/Users/giladram/Documents/NLP/Final Project/unified-summarization/data/finished_files/vocab'

vocab = Vocab(VOCAB_PATH, FLAGS.vocab_size)  # create a vocabulary

Finished constructing vocabulary of 29231 total words. Last word added: degustation


In [3]:
FLAGS.mode = 'train'
FLAGS.model = 'selector'

hparam_list = ['model', 'mode', 'eval_method', 'selector_loss_wt', 'inconsistent_loss', 'inconsistent_topk', 'lr',
               'adagrad_init_acc', 'rand_unif_init_mag', 'trunc_norm_init_std', 'max_grad_norm',
               'hidden_dim_selector', 'hidden_dim_rewriter', 'emb_dim', 'batch_size', 'max_art_len', 'max_sent_len',
               'max_dec_steps', 'max_enc_steps', 'coverage', 'cov_loss_wt', 'eval_gt_rouge', 'decode_method']
hps_dict = {}
for key, val in FLAGS.__flags.items():  # for each flag
    if key in hparam_list:  # if it's in the list
        hps_dict[key] = val.value  # add it to the dict
hps = namedtuple("HParams", hps_dict.keys())(**hps_dict)

In [4]:
batcher = Batcher(FLAGS.data_path, vocab, hps, single_pass=True)


INFO:tensorflow:reading file /Users/giladram/Documents/NLP/Final Project/unified-summarization/data/finished_files/chunked/train_000.bin


In [5]:
batcher.next_batch()

In [6]:
model = ExtractorModel(hps, vocab)
run_selector.setup_training(model, batcher)

INFO:tensorflow:Building graph...



Writing word embedding metadata file to train/vocab_metadata.tsv...

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the con

In [ ]:
run_selector.run_eval(model, batcher)